## Building NN for speech recogniton and optimize result 

In [92]:
import numpy as np
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
%matplotlib inline

from utils import *

### Get data

In [91]:
# read data
features_df = pd.read_csv('data.csv')

In [86]:
# get train label and data
all_labels = features_df.values[:,0]
x_data = features_df.values[:,1:]

In [88]:
# prepare categorical target values (e.g [0,0,0,1,0])
target = np.zeros((len(all_labels),41),dtype=int)
for i,_ in enumerate(all_labels):
    target[i][int(_)]=1

## Create train and test dataset

In [89]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.15, random_state=0)

for train_index, test_index in sss.split(x_data,target):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    

print('Size of training matrix:', X_train.shape)
print('Size of testing matrix:', X_test.shape)

Size of training matrix: (18217, 2808)
Size of testing matrix: (3215, 2808)


#### Normalize values

In [350]:
# normalize train
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)

In [357]:
# normalize test
scaler = preprocessing.StandardScaler().fit(X_test)
X_test=scaler.transform(X_test)

In [351]:
#get number of columns in training data
n_cols = x_data.shape[1]

In [352]:
model = Sequential()

#add layers to model
model.add(Dense(200, activation='sigmoid', input_shape=(n_cols,)))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(41, activation='softmax'))

In [353]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 200)               561800    
_________________________________________________________________
dense_46 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_47 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_48 (Dense)             (None, 41)                8241      
Total params: 650,441
Trainable params: 650,441
Non-trainable params: 0
_________________________________________________________________


In [354]:
early_stopping_monitor = EarlyStopping(patience=3)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [356]:
#train model
model.fit(X_train, y_train, epochs=50, batch_size=20,validation_split=0.2)

Train on 14574 samples, validate on 3644 samples
Epoch 1/50
14574/14574 [==============================] - 14s 955us/step - loss: 1.3543 - acc: 0.7305 - val_loss: 0.2554 - val_acc: 0.9608
Epoch 2/50
14574/14574 [==============================] - 13s 911us/step - loss: 0.1247 - acc: 0.9829 - val_loss: 0.0933 - val_acc: 0.9849
Epoch 3/50
14574/14574 [==============================] - 13s 871us/step - loss: 0.0363 - acc: 0.9962 - val_loss: 0.0707 - val_acc: 0.9846
Epoch 4/50
14574/14574 [==============================] - 13s 868us/step - loss: 0.0142 - acc: 0.9990 - val_loss: 0.0500 - val_acc: 0.9885
Epoch 5/50
14574/14574 [==============================] - 13s 873us/step - loss: 0.0069 - acc: 0.9996 - val_loss: 0.0490 - val_acc: 0.9877
Epoch 6/50
14574/14574 [==============================] - 13s 875us/step - loss: 0.0042 - acc: 0.9997 - val_loss: 0.0432 - val_acc: 0.9885
Epoch 7/50
14574/14574 [==============================] - 13s 882us/step - loss: 0.0028 - acc: 0.9999 - val_loss: 0.0

### Evaluate model

In [358]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 98.44%
